# T5 inference用コード

- [学習コード](https://colab.research.google.com/drive/1zDHDNQiEmkuPox_iyzS3XN3mAoc5hU2u?usp=sharing)

- setup.py
  - モデルのパラメータやディレクトリパスの設定
- calcSim.py
  - generate_summary
    - 記事の本文からタイトル(要約文)を生成
  - calc_score
    - rougeスコアの計算
  - make_output
    -  上記2関数をまとめたもの

In [1]:
from setup import *
from calcSim import *

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer


import warnings
warnings.simplefilter('ignore')

/Users/haru/opt/anaconda3/envs/t5/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## modelの読み込み、tokenizerの読み込み,スコア計算用オブジェクトの作成

In [2]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True,model_max_length=args.max_input_length)
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

## テスト記事
- https://news.yahoo.co.jp/articles/fde639ad7b20d7d2777ab801f40f4a1ec6889bd4

In [3]:
target = "「韓国代表株」のサムスン電子、連日株価下落…証券会社の見通しも「調整」"
input = "サムスン電子の株価が連日下落を続けている。２０日の終値は前営業日より１１００ウォン（１．８４％）の安値となる５万８７００ウォンを記録した。この２営業日連続で５万ウォン台となった。マクロ経済環境が悪化したことで市場では株価上昇幅を調整する動きが現れている。 金融情報会社のＦｎガイドによると、サムスン電子の４－６月期売り上げ見通しは７８兆１８２２億ウォン、営業利益見通しは１５兆２１２４億ウォンだった。前年同期比で売り上げは２２．８％、営業利益は２１．１％増えた数値だ。売り上げはこれまでの四半期売り上げで最も大きい規模だ。年間基準では売り上げが前年比１６．３％増の３２５兆２６１７億ウォン、営業利益が２１．７％増の６２兆８３２１億ウォンで、いずれも過去最高水準だ。 ＤＢ金融投資のオ・ギュジン研究員は「下半期の世界的なインフレにともなうパソコンとモバイルなどセット部門の不振でメモリー価格が下落するとみられ、短期的な懸念はあるが業績は良好なものとみられる」と話した。世界情勢の影響で５２週安値を記録しているが、現在の株価は厳しい業況をあらかじめ反映しているという説明だ。 オ研究員は、サムスン電子の４－６月期の営業利益を１４兆９０００億ウォンとし、このうち半導体部門が１０兆３０００億ウォンを占めるだろうと予想した。データセンター用の需要増加によりメモリーの出荷が増え、価格も善戦し半導体の業績は好調を見せるだろうと説明した。市場調査会社ＤＲＡＭエクスチェンジによると、先月のＤＲＡＭ（ＰＣ用ＤＤＲ４　８Ｇｂ基準）固定取引価格は３．３５ドルで、前月の３．４１ドルに比べ１．８％下落したが、市場予想値の３～５％より下げ幅は小さかった。 スマートフォンとＰＣ・テレビ事業は振るわないものとみられる。有進投資証券のイ・スンウ研究員は「モバイルエクスペリエンス（ＭＸ）事業部は出荷量減少にともなう売り上げ下落、部品価格とドル高の影響で業績悪化が避けられず、テレビと家電なども需要低下とコスト負担増加から抜け出しにくいだろう」と分析した。イ研究員はサムスン電子の４－６月期の営業利益推定値をこれまでの１６兆３０００億ウォンから１５兆ウォンに引き下げた。 その上で「サムスン電子は１－３月期の業績説明会で自信ある姿を見せたが、その間にインフレが高まり米国の０．７５％の大幅な政策金利引き上げが２８年ぶりに現実となった。現在のマクロ経済環境は前代未聞の状況」と指摘した。当分は上昇を見込みにくい状況という話だ。 Ｆｎガイドによると、証券会社はこの日サムスン電子の平均適正株価をこれまでの９万３６１９ウォンから９万１３８１ウォンに調整した。 一方、サムスン電子は２１日から主要経営陣と役員、海外法人長などが参加する上半期経営戦略会議を開く。デバイスエクスペリエンス（ＤＸ）部門は２１～２３日に水原（スウォン）本社で、半導体（ＤＳ）部門は２７～２９日に華城（ファソン）事業所でだ。"

## タイトル生成、類似度計算

In [4]:
cs = CalcSim(trained_model,tokenizer,args.max_input_length,args.max_target_length)
result = cs.make_output(input,target)

## 出力

In [5]:
result

0.6938502453120106